In [29]:
import pandas as pd
import numpy as np


In [88]:
df = pd.read_csv("data/2008.csv")

In [89]:
np.sort(df['DayofMonth'].unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])

In [90]:
dfa = pd.read_csv("data/airline_codes.csv")
dfa[dfa.code == 'MQ']['flight']

657    American Eagle Airlines
Name: flight, dtype: object

In [91]:
import datetime

df = df.sort(['Month'], ascending=[1])
df['Month'] = df['Month'].apply(lambda x: datetime.date(2008, x, 1).strftime('%b-%y'))
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2008,Jan-08,3,4,2003,1955,2211,2225,WN,335,...,4,8,0,NaN,0,NaN,NaN,NaN,NaN,NaN
403840,2008,Jan-08,3,4,1146,1130,1233,1220,MQ,4061,...,3,12,0,NaN,0,NaN,NaN,NaN,NaN,NaN
403841,2008,Jan-08,4,5,1152,1130,1250,1220,MQ,4061,...,2,26,0,NaN,0,22,0,8,0,0
403842,2008,Jan-08,5,6,1132,1130,1243,1220,MQ,4061,...,3,34,0,NaN,0,0,0,23,0,0
403843,2008,Jan-08,6,7,NaN,1130,NaN,1220,MQ,4061,...,NaN,NaN,1,B,0,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandasql

delays_sql = """
    select Month, SUM(DepDelay) As DepDelay, (sum(DepDelay) + sum(ArrDelay) + sum(LateAircraftDelay) + 
            sum(NASDelay) + sum(WeatherDelay) + sum(CarrierDelay)) as delays
    from df
    group by Month
    """

month = pandasql.sqldf(delays_sql.lower(), locals())
month

In [20]:
delays = df.groupby(['Month'])['DepDelay', 'ArrDelay', 
                                       'LateAircraftDelay', 
                                       'SecurityDelay', 
                                       'NASDelay',
                                       'WeatherDelay',
                                       'CarrierDelay'].mean()

delays.to_csv('data/2008delays.csv')

In [6]:
df.to_csv('data/2008.tsv', sep='\t')

In [7]:
df['DayofMonth'].unique()

array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,  2,  1])

In [182]:
delays_airport = df.groupby(['Origin'])['DepDelay', 'ArrDelay', 
                                       'LateAircraftDelay', 
                                       'SecurityDelay', 
                                       'NASDelay',
                                       'WeatherDelay',
                                       'CarrierDelay'].mean()

delays_airport.fillna(0)

delays_airport = delays_airport.sort(['DepDelay'], ascending=False)

delays_airport.head()

,DepDelay,ArrDelay,LateAircraftDelay,SecurityDelay,NASDelay,WeatherDelay,CarrierDelay
Origin,,,,,,,
ACK,29.854415,28.603819,24.432099,0.000000,32.993827,7.567901,19.228395
PUB,27.000000,74.000000,0.000000,0.000000,47.000000,27.000000,0.000000
CEC,24.186207,24.670949,23.938303,0.000000,41.226221,2.017995,11.123393
PIR,22.800000,23.800000,0.000000,0.000000,19.000000,0.000000,99.000000
SPI,22.314516,21.950673,24.331361,0.236686,49.000000,0.742604,19.899408


In [10]:
delays_airport.to_csv('data/2008delays_air.tsv', sep = "\t")

# Data for Chord

In [29]:
import pandasql

delays_sql = """
    select Origin, 
           Dest, 
           (avg(DepDelay) + avg(ArrDelay) + avg(LateAircraftDelay) + 
            avg(NASDelay) + avg(WeatherDelay) + avg(CarrierDelay)) as delays
    from df
    group by Origin, Dest
    """

orig_dest_delay = pandasql.sqldf(delays_sql.lower(), locals())

orig_dest_delay['delays'] = orig_dest_delay['delays'].fillna(0)

#orig_dest_delay = orig_dest_delay.sort(['delays'], ascending=0)
orig_dest_delay.head()

,Origin,Dest,delays
0,ABE,ATL,91.610286
1,ABE,BHM,0.000000
2,ABE,CLE,42.970444
3,ABE,CLT,35.327595
4,ABE,CVG,38.452412


In [101]:
#https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States
#airports = ['ATL', 'LAX', 'ORD', 'DFW', 'JFK', 'DEN', 'SFO',
#            'CLT', 'LAS', 'PHX', 'IAH', 'MIA', 'SEA', 'EWR',
#            'MCO', 'MSP', 'DTW', 'BOS', 'PHL', 'LGA']

airports = df['Origin'].unique()
airports

array(['IAD', 'IND', 'ISP', 'JAN', 'JAX', 'LAS', 'LAX', 'LBB', 'LIT',
       'MAF', 'MCI', 'MCO', 'MDW', 'MHT', 'MSY', 'OAK', 'OKC', 'OMA',
       'ONT', 'ORF', 'PBI', 'PDX', 'PHL', 'PHX', 'PIT', 'PVD', 'RDU',
       'RNO', 'RSW', 'SAN', 'SAT', 'SDF', 'SEA', 'SFO', 'SJC', 'SLC',
       'SMF', 'SNA', 'STL', 'TPA', 'TUL', 'TUS', 'ABQ', 'ALB', 'AMA',
       'AUS', 'BDL', 'BHM', 'BNA', 'BOI', 'BUF', 'BUR', 'BWI', 'CLE',
       'CMH', 'CRP', 'DAL', 'DEN', 'DTW', 'ELP', 'FLL', 'GEG', 'HOU',
       'HRL', 'ROC', 'DAY', 'ORD', 'EWR', 'SYR', 'IAH', 'LFT', 'MKE',
       'CHS', 'LCH', 'CLT', 'BTR', 'CRW', 'FAT', 'COS', 'MRY', 'LGB',
       'BFL', 'EUG', 'ICT', 'MEM', 'LGA', 'DCA', 'BTV', 'GRK', 'BRO',
       'TYS', 'DSM', 'BPT', 'GPT', 'GRR', 'PWM', 'MSP', 'RIC', 'CVG',
       'SAV', 'SRQ', 'GSO', 'CHA', 'XNA', 'GSP', 'LEX', 'MFE', 'ABE',
       'MLU', 'MOB', 'LRD', 'SHV', 'TLH', 'CAE', 'AEX', 'ATL', 'DFW',
       'BGR', 'AVL', 'BOS', 'MSN', 'HSV', 'MGM', 'MYR', 'VPS', 'CLL',
       'PNS', 'MTJ',

In [102]:
import random
import pprint

airports = map(lambda x: (x, "#%06x" % random.randint(0, 0xFFFFFF)), airports)

with open('airliners.csv', 'w') as a:
    a.write("name,color\n")
    count = 0
    for i in airports:
        a.write("{}, {}\n".format(i[0], i[1]))
        count += 1
        
len(airports)

303

In [ ]:
from pandas import *

matrix = []
for o in airports:
    ll = []
    for d in airports:
        dl = orig_dest_delay[(orig_dest_delay.Origin == o[0]) & (orig_dest_delay.Dest == d[0])]['delays']
        if dl.any():
            ll = ll + [dl.mean()]
        else:
            ll = ll + [0]
    matrix = matrix + [ll]
    
DataFrame(matrix)   


### Cancellations

In [254]:
df['CancellationCode'] = df['CancellationCode'].fillna('E')
np.sort(df['CancellationCode'].unique())

array(['A', 'B', 'C', 'D', 'E'], dtype=object)

In [255]:
c = df[df.Cancelled == 1][['Month','UniqueCarrier', 'Distance', 'Cancelled', 'CancellationCode']]
print len(c)
c.head()

137434


,Month,UniqueCarrier,Distance,Cancelled,CancellationCode
403843,Jan-08,MQ,174,1,B
403837,Jan-08,MQ,223,1,C
403828,Jan-08,MQ,223,1,C
403835,Jan-08,MQ,223,1,C
403886,Jan-08,MQ,473,1,C


In [256]:
dow_sql = """
    select Month, UniqueCarrier, COUNT(*) as "Total Flights"
    from c
    group by Month, UniqueCarrier
    """

t = pandasql.sqldf(dow_sql, locals())


print len(t)
t.head()

230


,Month,UniqueCarrier,Total Flights
0,Apr-08,9E,514
1,Apr-08,AA,3918
2,Apr-08,AS,158
3,Apr-08,B6,127
4,Apr-08,CO,87


In [257]:
dow_sql = """
    select Month, UniqueCarrier, avg(Distance) as Distance, SUM(Cancelled) as Cancelled, CancellationCode
    from c
    group by Month, UniqueCarrier, CancellationCode
    """

d = pandasql.sqldf(dow_sql, locals())

print len(d)
d.head()

640


,Month,UniqueCarrier,Distance,Cancelled,CancellationCode
0,Apr-08,9E,479.567775,391,A
1,Apr-08,9E,465.152542,59,B
2,Apr-08,9E,443.968750,64,C
3,Apr-08,AA,900.115530,3696,A
4,Apr-08,AA,781.068702,131,B


In [258]:
d = d.merge(t, on=['Month','UniqueCarrier'])
d.head()

,Month,UniqueCarrier,Distance,Cancelled,CancellationCode,Total Flights
0,Apr-08,9E,479.567775,391,A,514
1,Apr-08,9E,465.152542,59,B,514
2,Apr-08,9E,443.968750,64,C,514
3,Apr-08,AA,900.115530,3696,A,3918
4,Apr-08,AA,781.068702,131,B,3918


In [259]:
d['CancellationCode'] = d['CancellationCode'].apply(lambda x: "Carrier Delay" if x == 'A' else x)
d['CancellationCode'] = d['CancellationCode'].apply(lambda x: "Weather Delay" if x == 'B' else x)
d['CancellationCode'] = d['CancellationCode'].apply(lambda x: "National Airspace System Delay" if x == 'C' else x)
d['CancellationCode'] = d['CancellationCode'].apply(lambda x: "Security Delay" if x == 'D' else x)

dic = dfa[['code', 'flight']].set_index('code').to_dict()
d['UniqueCarrier'] = d['UniqueCarrier'].apply(lambda x: dic['flight'][x])

d['Percentage Cancelled'] = d['Cancelled'] / d['Total Flights']

#A = carrier, B = weather, C = NAS, D = security
d.head()

,Month,UniqueCarrier,Distance,Cancelled,CancellationCode,Total Flights,Percentage Cancelled
0,Apr-08,Pinnacle Airlines,479.567775,391,Carrier Delay,514,0.760700
1,Apr-08,Pinnacle Airlines,465.152542,59,Weather Delay,514,0.114786
2,Apr-08,Pinnacle Airlines,443.968750,64,National Airspace System Delay,514,0.124514
3,Apr-08,American Airlines,900.115530,3696,Carrier Delay,3918,0.943338
4,Apr-08,American Airlines,781.068702,131,Weather Delay,3918,0.033435


In [260]:
d.to_csv('data/2008cancelled.csv')

In [262]:
len(d['UniqueCarrier'].unique())

20